## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Varnavino,RU,57.4035,45.0916,4.26,98,70,4.97,broken clouds
1,1,Saint-Philippe,RE,-21.3585,55.7679,80.19,85,93,3.47,light rain
2,2,Hermanus,ZA,-34.4187,19.2345,63.90,68,98,9.60,overcast clouds
3,3,Torbay,CA,47.6666,-52.7314,41.29,95,100,5.01,heavy intensity rain
4,4,Narsaq,GL,60.9167,-46.0500,25.02,71,100,12.93,snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Saint-Philippe,RE,-21.3585,55.7679,80.19,85,93,3.47,light rain
6,6,Belmonte,BR,-15.8631,-38.8828,81.10,77,91,6.55,overcast clouds
8,8,Trat,TH,12.5000,102.5000,81.99,59,96,3.94,overcast clouds
10,10,Puerto Ayora,EC,-0.7393,-90.3518,82.36,74,94,9.37,overcast clouds
18,18,Sinnamary,GF,5.3833,-52.9500,77.07,88,70,7.74,broken clouds
20,20,East London,ZA,-33.0153,27.9116,71.51,88,74,16.06,light rain
21,21,Merauke,ID,-8.4667,140.3333,76.80,93,100,8.30,overcast clouds
29,29,Kapaa,US,22.0752,-159.3190,81.12,71,1,1.99,clear sky
30,30,Micheweni,TZ,-4.9667,39.8333,80.76,82,67,11.63,light rain
31,31,Luderitz,NaN,-26.6481,15.1594,72.27,70,0,20.04,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Saint-Philippe,RE,-21.3585,55.7679,80.19,85,93,3.47,light rain
6,6,Belmonte,BR,-15.8631,-38.8828,81.10,77,91,6.55,overcast clouds
8,8,Trat,TH,12.5000,102.5000,81.99,59,96,3.94,overcast clouds
10,10,Puerto Ayora,EC,-0.7393,-90.3518,82.36,74,94,9.37,overcast clouds
18,18,Sinnamary,GF,5.3833,-52.9500,77.07,88,70,7.74,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Saint-Philippe,RE,80.19,light rain,-21.3585,55.7679,
6,Belmonte,BR,81.10,overcast clouds,-15.8631,-38.8828,
8,Trat,TH,81.99,overcast clouds,12.5000,102.5000,
10,Puerto Ayora,EC,82.36,overcast clouds,-0.7393,-90.3518,
18,Sinnamary,GF,77.07,broken clouds,5.3833,-52.9500,
20,East London,ZA,71.51,light rain,-33.0153,27.9116,
21,Merauke,ID,76.80,overcast clouds,-8.4667,140.3333,
29,Kapaa,US,81.12,clear sky,22.0752,-159.3190,
30,Micheweni,TZ,80.76,light rain,-4.9667,39.8333,
37,Sao Filipe,CV,73.98,broken clouds,14.8961,-24.4956,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found. Skipping...")

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Saint-Philippe,RE,80.19,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
6,Belmonte,BR,81.10,overcast clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
8,Trat,TH,81.99,overcast clouds,12.5000,102.5000,สวนหนิง-อัศ
10,Puerto Ayora,EC,82.36,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
18,Sinnamary,GF,77.07,broken clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))